In [1]:
import pickle as pkl
# import pandas as pd
# import yfinance as yf
# import time

# tickers = [line.rstrip("\n") for line in open("sp500_constituents.txt", "r").readlines()]

# # Yahoo uses BRK-B instead of BRK.B, same for BF-B -> BF.B
# tickers = [t.replace(".", "-") for t in tickers]


# def fetch_history(tickers, start, end=None, interval="1d"):
#     all_data = []
#     chunk_size = 50  # <= 100 is safer

#     for i in range(0, len(tickers), chunk_size):
#         batch = tickers[i : i + chunk_size]
#         print(f"Fetching {batch[0]}...{batch[-1]}")

#         df = yf.download(
#             tickers=batch,
#             start=start,
#             end=end,
#             interval=interval,
#             group_by="ticker",
#             auto_adjust=False,
#             threads=True,
#         )
#         all_data.append(df)

#         time.sleep(1.5)  # avoid being throttled

#     return all_data


# history_batches = fetch_history(tickers, start="2000-01-01")
# pkl.dump(history_batches, open("./sp500.pkl", "wb"))
# # history_batches = pkl.load(open("./sp500.pkl", "rb"))

# history = pd.concat(history_batches, axis=1)

# close_df = history.xs("Close", axis=1, level=1)

# open_df = history.xs("Open", axis=1, level=1)
# high_df = history.xs("High", axis=1, level=1)
# low_df = history.xs("Low", axis=1, level=1)
# vol_df = history.xs("Volume", axis=1, level=1)

# pkl.dump(close_df, open("./sp500_close.pkl", "wb"))

In [2]:
close_df = pkl.load(open("./sp500_close.pkl", "rb"))
close_df = close_df.dropna(axis=1, how="any")
dates = close_df.index.values

In [3]:
# Convert to polars for your pipeline
import polars as pl
from backtest_lib.market.polars_impl import Axis

close_pl = pl.from_pandas(close_df)
securities = close_pl.columns
axis = Axis.from_names(securities)

In [4]:
pastview = close_pl.transpose()

pastview = pastview.rename(
    {orig: orig.replace("column_", "period_") for orig in pastview.columns}
)

window_size = 4
stagger = 0

pastview.select(pastview.columns[stagger : window_size + stagger])

period_0,period_1,period_2,period_3
f64,f64,f64,f64
150.240005,144.419998,136.149994,136.229996
170.404495,167.522003,164.356995,163.253998
648.049988,623.5,578.840027,567.619995
135.419998,135.160004,135.869995,135.229996
129.360001,132.399994,129.309998,128.070007
…,…,…,…
132.020004,133.820007,133.800003,130.979996
52.439999,53.470001,54.02,53.759998
583.900024,587.599976,558.179993,555.159973


In [5]:
# now with the lib
from backtest_lib.market.polars_impl import PolarsPastView
from backtest_lib.market import PastView

close_prices_df = close_pl.with_columns(pl.Series("date", dates))
past_cost_prices = PolarsPastView.from_data_frame(close_prices_df)
print(isinstance(past_cost_prices, PastView))
print(past_cost_prices.by_period[-1]["AAPL"])

True
230.88999938964844


In [6]:
print(past_cost_prices.by_security["AAPL"])

PolarsTimeseries(_vec=shape: (909,)
Series: 'AAPL' [f64]
[
	182.009995
	179.699997
	174.919998
	172.0
	172.169998
	…
	229.649994
	233.330002
	232.779999
	231.589996
	230.889999
], _axis=PeriodAxis(dt64=array(['2022-01-03T00:00:00.000000000', '2022-01-04T00:00:00.000000000',
       '2022-01-05T00:00:00.000000000', '2022-01-06T00:00:00.000000000',
       '2022-01-07T00:00:00.000000000', '2022-01-10T00:00:00.000000000',
       '2022-01-11T00:00:00.000000000', '2022-01-12T00:00:00.000000000',
       '2022-01-13T00:00:00.000000000', '2022-01-14T00:00:00.000000000',
       '2022-01-18T00:00:00.000000000', '2022-01-19T00:00:00.000000000',
       '2022-01-20T00:00:00.000000000', '2022-01-21T00:00:00.000000000',
       '2022-01-24T00:00:00.000000000', '2022-01-25T00:00:00.000000000',
       '2022-01-26T00:00:00.000000000', '2022-01-27T00:00:00.000000000',
       '2022-01-28T00:00:00.000000000', '2022-01-31T00:00:00.000000000',
       '2022-02-01T00:00:00.000000000', '2022-02-02T00:00:00.0000000

In [7]:
from backtest_lib.market import BySecurity, ByPeriod


print(
    isinstance(past_cost_prices.by_security, BySecurity),
    isinstance(past_cost_prices.by_period, ByPeriod),
)


prices = past_cost_prices.by_period[-1]
prices_plus_one = past_cost_prices.by_period[-1] + 1
assert (prices_plus_one - prices).sum() == len(prices)
assert past_cost_prices.by_period[-1]._scalar_type is float
assert past_cost_prices.by_period[-1].floor()._scalar_type is int
assert (past_cost_prices.by_period[-1].floor() + 5 + 2.5 + 2)._scalar_type is float
assert (past_cost_prices.by_period[-1].floor() + 5 + 2 + 2)._scalar_type is int

True True


In [8]:
assert (
    past_cost_prices.by_security[["MSFT", "AAPL"]]
    .by_period[2:4]
    .by_period.as_df()
    .equals(
        past_cost_prices.by_period[2:4].by_security[["MSFT", "AAPL"]].by_period.as_df()
    )
)

In [9]:
import random
from time import perf_counter
from backtest_lib.market.polars_impl import SeriesUniverseMapping
from backtest_lib.portfolio import QuantityPortfolio

import numpy as np

secs = tuple(past_cost_prices.by_period[-1].keys())
qtys = [random.randint(0, 10) for sec in secs]

holdings = SeriesUniverseMapping.from_vectors(secs, qtys)

pf = QuantityPortfolio(cash=0, holdings=holdings)

prices = past_cost_prices.by_period[-1]

wpf = pf.into_weighted(prices)
t0 = perf_counter()
for _ in range(1000):
    wpf = pf.into_weighted(prices)
print(f"{perf_counter() - t0}")

0.3844159000000218


In [10]:
from backtest_lib.strategy import (
    MarketView,
    Decision,
)
from backtest_lib.portfolio import WeightedPortfolio
from backtest_lib.strategy.context import StrategyContext
from backtest_lib.universe import Universe, PastUniversePrices
from backtest_lib.backtest import Backtest, BacktestSettings
from backtest_lib.market.polars_impl import SeriesUniverseMapping

market_view = MarketView(
    prices=PastUniversePrices(close=past_cost_prices), periods=dates
)
securities = tuple(securities)
universe = securities
initial_portfolio = WeightedPortfolio(
    cash=0,
    holdings=SeriesUniverseMapping.from_names_and_data(
        universe, pl.Series(np.tile(1 / len(securities), len(securities)))
    ),
)

target_portfolio = initial_portfolio


def strategy(
    universe: Universe,
    market: MarketView,
    current_portfolio: WeightedPortfolio,
    ctx: StrategyContext,
):
    lookback_periods = 120
    if len(market.periods) < lookback_periods:
        return Decision(initial_portfolio)

    prices_120_periods_ago = market.prices.close.by_period[-(lookback_periods)]
    prices_now = market.prices.close.by_period[-1]
    momentum_mapping = (prices_now / prices_120_periods_ago) - 1
    stonks_with_posi_mom = [x for x, y in momentum_mapping.items() if y >= 0]
    stonks_with_neg_mom = [x for x, y in momentum_mapping.items() if y < 0]

    if len(stonks_with_neg_mom) == 0 or len(stonks_with_posi_mom) == 0:
        # do nothing
        return Decision(current_portfolio)

    momentum_weight_pos = 0.1 / len(stonks_with_posi_mom)
    momentum_weight_neg = 0.1 / len(stonks_with_neg_mom)

    portfolio_changes = {
        **{sec: momentum_weight_pos for sec in stonks_with_posi_mom},
        **{sec: -momentum_weight_neg for sec in stonks_with_neg_mom},
    }

    new_target_holdings = current_portfolio.holdings + portfolio_changes

    new_target_portfolio = WeightedPortfolio(
        current_portfolio.cash, new_target_holdings
    )

    return Decision(new_target_portfolio)


settings = BacktestSettings(allow_short=False)

bt = Backtest(
    strategy=strategy,
    universe=universe,
    market_view=market_view,
    initial_portfolio=initial_portfolio,
    settings=settings,
)


results = bt.run()

print(f"total return: {(results.total_growth - 1) * 100:.2f}%")

end_port_df = pl.DataFrame(
    {
        "Security": bt._current_portfolio.holdings.keys(),
        "Weight": bt._current_portfolio.holdings.values(),
    }
)
print(f"end portfolio weights: {end_port_df}")
sorted_weights = sorted(
    bt._current_portfolio.holdings.items(), key=lambda x: x[1], reverse=True
)
print("sorted weights:", sorted_weights)

total return: 23.90%
end portfolio weights: shape: (497, 2)
┌──────────┬──────────┐
│ Security ┆ Weight   │
│ ---      ┆ ---      │
│ str      ┆ f64      │
╞══════════╪══════════╡
│ AMD      ┆ 0.004505 │
│ AMZN     ┆ 0.000918 │
│ ALGN     ┆ 0.0      │
│ ABBV     ┆ 0.00219  │
│ ALLE     ┆ 0.004491 │
│ …        ┆ …        │
│ UHS      ┆ 0.0      │
│ VZ       ┆ 0.004505 │
│ ZBRA     ┆ 0.00116  │
│ ZBH      ┆ 0.0      │
│ ZTS      ┆ 0.0      │
└──────────┴──────────┘
sorted weights: [('EBAY', 0.005019339580333431), ('IDXX', 0.004781609264868832), ('PLTR', 0.004730462244289312), ('NEM', 0.004713035583124597), ('HCA', 0.004671816899051064), ('MO', 0.00465080664676583), ('MNST', 0.004618301956038806), ('STE', 0.004599289357491079), ('EA', 0.00459389967078288), ('HAS', 0.004591459111150322), ('BA', 0.004568587001635208), ('FAST', 0.004564732244013903), ('AVGO', 0.0045626692350303425), ('JNJ', 0.004561146720907493), ('ECL', 0.004560752503815842), ('ATO', 0.004557566698764542), ('GILD', 0.004554

In [11]:
# for x, y in initial_portfolio.holdings.items():
#     print(f"{x},{y}")

rounded = [round(val, 6) for val in initial_portfolio.holdings.values()]

total_value = 1_000_000

quantities_with_1m = initial_portfolio.into_quantities(prices, total_value)
assert (
    quantities_with_1m.holdings * prices
).sum() + quantities_with_1m.cash == total_value

In [12]:
(initial_portfolio.holdings + {"AAPL": 0.1}).sum()

1.100000000000001

In [13]:
past_cost_prices.by_period[0]

SeriesUniverseMapping(names=('AMD', 'AMZN', 'ALGN', 'ABBV', 'ALLE', 'AWK', 'ANET', 'AMT', 'APTV', 'GOOGL', 'ADBE', 'AMAT', 'APO', 'A', 'APA', 'AON', 'T', 'APD', 'ADM', 'AFL', 'AJG', 'AEP', 'AMGN', 'AXP', 'MO', 'ABT', 'LNT', 'ALL', 'ATO', 'AAPL', 'ARE', 'MMM', 'AME', 'AIZ', 'AMP', 'AES', 'ADSK', 'AKAM', 'ACGL', 'ABNB', 'AMCR', 'ADI', 'GOOG', 'ADP', 'APH', 'AEE', 'ACN', 'AOS', 'AIG', 'ALB', 'KMX', 'BIIB', 'CZR', 'BLDR', 'CBOE', 'BSX', 'CBRE', 'XYZ', 'AVGO', 'CDW', 'AZO', 'BXP', 'BMY', 'BK', 'BX', 'BG', 'COR', 'BR', 'BAC', 'CARR', 'TECH', 'BA', 'CAH', 'BLK', 'BAX', 'CPB', 'CVX', 'BF-B', 'CNP', 'BBY', 'BALL', 'CF', 'CAT', 'CPT', 'BKNG', 'CDNS', 'BRK-B', 'AVY', 'AXON', 'CNC', 'COF', 'CCL', 'CHRW', 'CHTR', 'SCHW', 'CRL', 'AVB', 'BKR', 'BRO', 'BDX', 'DVA', 'CPAY', 'DXCM', 'DDOG', 'CTSH', 'DAY', 'CSGP', 'CMG', 'CTVA', 'FANG', 'C', 'DG', 'COO', 'CINF', 'DELL', 'CMI', 'DECK', 'CTAS', 'CB', 'CCI', 'CTRA', 'CMCSA', 'CRWD', 'COP', 'CFG', 'ED', 'CVS', 'CAG', 'KO', 'GLW', 'DE', 'DLR', 'DHR', 'CSCO', 

In [14]:
neg_series = bt._current_portfolio.holdings.as_series()
neg_mass = neg_series.clip(upper_bound=0).sum()
pos_mass = neg_series.clip(lower_bound=0).sum()
pos_only_series = neg_series.clip(lower_bound=0)

# l = []

# for val in pos_only_series:
#     l.append()

bt._current_portfolio.into_long_only()

WeightedPortfolio(cash=0.0, holdings=SeriesUniverseMapping(names=('AMD', 'AMZN', 'ALGN', 'ABBV', 'ALLE', 'AWK', 'ANET', 'AMT', 'APTV', 'GOOGL', 'ADBE', 'AMAT', 'APO', 'A', 'APA', 'AON', 'T', 'APD', 'ADM', 'AFL', 'AJG', 'AEP', 'AMGN', 'AXP', 'MO', 'ABT', 'LNT', 'ALL', 'ATO', 'AAPL', 'ARE', 'MMM', 'AME', 'AIZ', 'AMP', 'AES', 'ADSK', 'AKAM', 'ACGL', 'ABNB', 'AMCR', 'ADI', 'GOOG', 'ADP', 'APH', 'AEE', 'ACN', 'AOS', 'AIG', 'ALB', 'KMX', 'BIIB', 'CZR', 'BLDR', 'CBOE', 'BSX', 'CBRE', 'XYZ', 'AVGO', 'CDW', 'AZO', 'BXP', 'BMY', 'BK', 'BX', 'BG', 'COR', 'BR', 'BAC', 'CARR', 'TECH', 'BA', 'CAH', 'BLK', 'BAX', 'CPB', 'CVX', 'BF-B', 'CNP', 'BBY', 'BALL', 'CF', 'CAT', 'CPT', 'BKNG', 'CDNS', 'BRK-B', 'AVY', 'AXON', 'CNC', 'COF', 'CCL', 'CHRW', 'CHTR', 'SCHW', 'CRL', 'AVB', 'BKR', 'BRO', 'BDX', 'DVA', 'CPAY', 'DXCM', 'DDOG', 'CTSH', 'DAY', 'CSGP', 'CMG', 'CTVA', 'FANG', 'C', 'DG', 'COO', 'CINF', 'DELL', 'CMI', 'DECK', 'CTAS', 'CB', 'CCI', 'CTRA', 'CMCSA', 'CRWD', 'COP', 'CFG', 'ED', 'CVS', 'CAG', 'KO'